<a href="https://colab.research.google.com/github/Saif-M-Dhrubo/hierarchical-attention-HAR/blob/master/MEx_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###CUDA GPU Checking

In [1]:
!nvidia-smi

Mon Apr  6 07:42:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Library Import

In [2]:
import os
import requests
import zipfile
import glob
import platform

import time
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##MEx Dataset

###Routine for Fetching

In [0]:
def get_dataset(url:str, data_directory:str, file_name:str):

    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    print('GETTING DATASET ...')

    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    if not os.path.exists(file_name):
        response = requests.get(url, stream=True)
        data_file = open(file_name, 'wb')

        for chunk in response.iter_content(chunk_size=1024):
            data_file.write(chunk)

        data_file.close()
        print('\n---DATASET DOWNLOAD COMPLETE---')

###Data Download

In [0]:
DATA_URL = 'https://data.mendeley.com/datasets/p89fwbzmkd/3/files/227baf25-f0c0-4025-9c9b-9bda2743fbed/MEx.zip?dl=1'
DATA_DIRECTORY = '/'
DATA_FILE = 'MEx.zip'

In [5]:
get_dataset(DATA_URL, DATA_DIRECTORY, DATA_FILE)

2020-04-06 07:42:11
GETTING DATASET ...

---DATASET DOWNLOAD COMPLETE---


In [6]:
!unzip MEx.zip -d MEx_Data/

Archive:  MEx.zip
   creating: MEx_Data/act/
   creating: MEx_Data/act/01/
  inflating: MEx_Data/act/01/01_act_1.csv  
  inflating: MEx_Data/act/01/07_act_1.csv  
  inflating: MEx_Data/act/01/06_act_1.csv  
  inflating: MEx_Data/act/01/05_act_1.csv  
  inflating: MEx_Data/act/01/04_act_1.csv  
  inflating: MEx_Data/act/01/04_act_2.csv  
  inflating: MEx_Data/act/01/03_act_1.csv  
  inflating: MEx_Data/act/01/02_act_1.csv  
   creating: MEx_Data/act/02/
  inflating: MEx_Data/act/02/03_act_1.csv  
  inflating: MEx_Data/act/02/02_act_1.csv  
  inflating: MEx_Data/act/02/01_act_1.csv  
  inflating: MEx_Data/act/02/07_act_1.csv  
  inflating: MEx_Data/act/02/06_act_1.csv  
  inflating: MEx_Data/act/02/05_act_1.csv  
  inflating: MEx_Data/act/02/04_act_1.csv  
  inflating: MEx_Data/act/02/04_act_2.csv  
   creating: MEx_Data/act/03/
  inflating: MEx_Data/act/03/01_act_1.csv  
  inflating: MEx_Data/act/03/07_act_1.csv  
  inflating: MEx_Data/act/03/06_act_1.csv  
  inflating: MEx_Data/act/03/

In [0]:
DATA_DIR = 'MEx_Data'
THIGH_ACCEL = 'act'
WRIST_ACCEL = 'acw'

In [0]:
def quantize_time(timestamp, quantization_level=2):
    m_sec = str(timestamp.microsecond)
    q_msec = m_sec[:quantization_level]

    timestamp = timestamp.replace(microsecond=(int(q_msec) * (10 ** (6 - quantization_level))))

    return timestamp

In [0]:
def get_activity(file_name:str):
    activivity = int(file_name.strip().split('_')[0])

    return activivity

In [0]:
def get_merged_data():

    complete_df = pd.DataFrame()

    SUBJECT_LIST = ['%02d' % x for x in range(1, 31)]

    for subj in SUBJECT_LIST:
        accel_t_dir = os.path.join(DATA_DIR, THIGH_ACCEL, subj)
        accel_w_dir = os.path.join(DATA_DIR, WRIST_ACCEL, subj)

        accel_t_files = sorted(os.listdir(accel_t_dir))
        accel_w_files = sorted(os.listdir(accel_w_dir))

        for i in range(len(accel_t_files)):
            accel_t = os.path.join(accel_t_dir, accel_t_files[i])
            accel_w = os.path.join(accel_w_dir, accel_w_files[i])

            df_t = pd.read_csv(accel_t, header=None, names=['timestamp', 'act_x', 'act_y', 'act_z'])
            df_w = pd.read_csv(accel_w, header=None, names=['timestamp', 'acw_x', 'acw_y', 'acw_z'])

            df_t['timestamp'] = pd.to_datetime(df_t['timestamp'])
            df_w['timestamp'] = pd.to_datetime(df_w['timestamp'])

            df_t['timestamp'] = df_t['timestamp'].apply(quantize_time)
            df_w['timestamp'] = df_w['timestamp'].apply(quantize_time)

            df_t = df_t.groupby(['timestamp'], as_index=False)['act_x', 'act_y', 'act_z'].mean()
            df_w = df_w.groupby(['timestamp'], as_index=False)['acw_x', 'acw_y', 'acw_z'].mean()

            merged = pd.merge(df_t, df_w, how='outer', on=['timestamp', 'timestamp'], sort=True)
            merged['subject_id'] = int(subj)
            merged['activity'] = get_activity(accel_t_files[i])
            
            complete_df = pd.concat([complete_df, merged], ignore_index=True)

    complete_df = complete_df.sort_values(by='timestamp', ignore_index=True)

    return complete_df

In [12]:
data = get_merged_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [13]:
data.sample(15)

,timestamp,act_x,act_y,act_z,acw_x,acw_y,acw_z,subject_id,activity
622806,2019-02-14 11:47:08.590,-0.593750,0.750000,-0.187500,-0.875000,-0.156250,-0.531250,18,2
1182678,2019-03-26 16:43:23.180,-0.250000,-0.984375,0.078125,-0.171875,-0.734375,1.500000,30,6
1146661,2019-03-26 16:10:31.620,0.898438,-0.218750,0.203125,0.578125,0.265625,-0.828125,29,7
601572,2019-02-14 10:17:41.960,0.078125,-0.953125,0.078125,0.468750,0.484375,-0.765625,17,5
913743,2019-03-07 13:09:28.620,-0.835938,-0.632812,-0.304688,-0.578125,-0.062500,0.734375,24,2
1005881,2019-03-25 13:26:06.880,0.000000,-0.062500,-0.984375,-0.843750,0.453125,-0.406250,26,4
489813,2018-11-08 12:26:14.190,-1.093750,-0.031250,0.125000,0.843750,0.062500,-0.500000,13,7
455993,2018-11-08 12:13:22.510,0.453125,-0.250000,-0.890625,0.562500,0.109375,-0.843750,13,1
239857,2018-10-11 11:56:16.560,0.781250,-0.437500,0.359375,0.187500,0.203125,0.703125,8,7
1073852,2019-03-25 14:11:38.170,0.500000,0.703125,-0.375000,-0.500000,0.187500,0.765625,28,2


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229840 entries, 0 to 1229839
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   timestamp   1229840 non-null  datetime64[ns]
 1   act_x       1227493 non-null  float64       
 2   act_y       1227493 non-null  float64       
 3   act_z       1227493 non-null  float64       
 4   acw_x       1189207 non-null  float64       
 5   acw_y       1189207 non-null  float64       
 6   acw_z       1189207 non-null  float64       
 7   subject_id  1229840 non-null  int64         
 8   activity    1229840 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 84.4 MB


In [15]:
data.isna().sum()

timestamp         0
act_x          2347
act_y          2347
act_z          2347
acw_x         40633
acw_y         40633
acw_z         40633
subject_id        0
activity          0
dtype: int64